# Тема 28. Чат-боты

Обработка текстов нейронными сетями дает возможности, о которых раньше можно было только мечтать. Мы уже познакомились с некоторыми примерами задач и методов их решения. 

Среди задач обработки текстов особое место занимает задача создания чат-ботов - программ, которые могут понимать естественную речь человека (в основном письменную, но также и устную) и отвечать человеку правдоподобным образом.

Чат боты можно разделить на две категории:
* универсальные, которые предназначены для ведения беседы на общие темы (о погоде, фильмах, политике, экономике и т.п.). 
* специализированные, которые предназначены для ведения беседы на узкоспециализированные темы. Среди таких чат-ботов очень востребованы чат-боты для бизнеса, которые могут отвечать на вопросы клиентов. 

Сегодня мы реализуем пример специализированного чат-бота для клиентов ресторанов, который может ответить пользователю где найти подходящий ресторан.

# Специализированный чат-бот на платформе DeepPavlov

Сегодня нам нет нужды писать все-все необходимые функции для чат-ботов самостоятельно. Существует большое количество библиотек, в которых создание своего чат-бота стало достаточно простым.

Одна из таких библиотек - отечественная платформа [DeepPavlov](https://github.com/deepmipt/DeepPavlov), установим ее. На сегодняшний лень эта библиотека работает с tensorflow версии 1.x, и множество других известных нам библиотек (например spacy, gensim и д.р.), они будут заменены или установлены автоматически.

Готовится к выходу и другая отечественная библиотека [SOVA](https://sova.ai/).


In [2]:
# подавление предупреждений
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Установка
!pip install deeppavlov
!python -m deeppavlov install gobot_simple_dstc2

     |████████████████████████████████| 901 kB 7.7 MB/s 
     |████████████████████████████████| 46 kB 4.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 1.5 MB 32.4 MB/s 
     |████████████████████████████████| 2.9 MB 57.8 MB/s 
     |████████████████████████████████| 40 kB 24 kB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
     |████████████████████████████████| 3.8 MB 60.5 MB/s 
     |████████████████████████████████| 859 kB 50.9 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 
     |████████████████████████████████| 510 kB 50.0 MB/s 
     |████████████████████████████████| 654 kB 49.9 MB/s 
     |████████████████████████████████| 2.1 MB 35.5 MB/s 
     |████████████████████████████████| 10.4 MB 40.6 MB/s 
     |████████████████████████████████| 65 kB 5.2 MB/s 
     |████████████████████████████████| 8.2 MB 54.0 MB/s 
     |█████████████████████

2021-08-16 16:24:23.605 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'gobot_simple_dstc2' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/go_bot/gobot_simple_dstc2.json'
     |████████████████████████████████| 692 kB 8.0 MB/s 
     |████████████████████████████████| 12.0 MB 7.8 MB/s 
     |████████████████████████████████| 24.2 MB 64 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 110.5 MB 410 bytes/s 
     |████████████████████████████████| 503 kB 69.3 MB/s 
     |████████████████████████████████| 3.8 MB 51.0 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=4873d0a4eaaa7f32130d2b122701a83b0fc9033818fc2423d0162a3c363bc340
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f55

## Подготовка данных

Любая работа начинается с анализа и подготовки необходимых данных. 

Для обучения чат-бота мы будем пользоваться набором данных [Dialogue State Tracking Challenge 2 (DSTC-2)](http://camdial.org/~mh521/dstc/). В этом наборе представлены диалоги реальных людей и системы бронирования ресторана и к диалогу даны "слоты" и "действия".   

In [4]:
# загрузчик данных
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

# загружаем в директорию 'my_data'
data = SimpleDSTC2DatasetReader().read('my_data')

2021-08-16 16:29:59.557 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 269: [PosixPath('my_data/simple-dstc2-val.json'), PosixPath('my_data/simple-dstc2-tst.json')]]
2021-08-16 16:29:59.558 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 270: [downloading data from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data]
2021-08-16 16:29:59.563 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data/simple_dstc2.tar.gz
100%|██████████| 497k/497k [00:04<00:00, 115kB/s] 
2021-08-16 16:30:05.674 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting my_data/simple_dstc2.tar.gz archive into my_data
2021-08-16 16:30:05.721 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my_data/simple-dstc2-trn.json]
2021-08-16 16:30:05.857 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['ds

In [5]:
# проверим, что файлы загрузились
!ls my_data

simple-dstc2-templates.txt  simple-dstc2-tst.json
simple-dstc2-trn.json	    simple-dstc2-val.json


Обучающие (`simple-dstc2-trn.json`), проверочные (`simple-dstc2-val.json`) и тестовые (`simple-dstc2-tst.json`) данные хранятся в формате json, посмотрим на них.

In [ ]:
# посмотрим на несколько записей
!head -n 101 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },
    {
      "speaker": 1,
      "text": "any",
      "slots": [
        [
          "this",
          "dontcare"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What part of town do you have in mind?",
      "slots": [],
      "act": "request_area"
    },
    {
      "speaker": 1,
      "text": "south",
      "slots": [
        [
          "area",
          "south"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "api_call area=\"south\" food

Для получения пакетов данных подключим `DatasetIterator`.

In [6]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

Теперь можно получать пакеты данных с диалогами DSTC-2, в котором разделены запросы пользователя и ответы системы.

In [8]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'): # один пример диалога из обучающих данных
    turns_x, turns_y = dialog # запрос пользователя \ ответ системы
    
    print("Запрос пользователя:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nОтвет системы:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

Запрос пользователя:
----------------

[   {'prev_resp_act': None, 'text': ''},
    {'prev_resp_act': 'welcomemsg', 'text': 'id like to find a restaurant'},
    {'prev_resp_act': 'request_pricerange', 'text': 'in appleton wiscon'},
    {   'db_result': {   'addr': '88 mill road city centre',
                         'area': 'centre',
                         'food': 'chinese',
                         'name': 'rice house',
                         'phone': '01223 367755',
                         'pricerange': 'cheap'},
        'prev_resp_act': 'api_call',
        'text': 'in appleton wiscon'},
    {'prev_resp_act': 'offer_name', 'text': 'restaurant'},
    {   'prev_resp_act': 'offer_name',
        'slots': [['food', 'irish']],
        'text': 'irish food'},
    {   'db_result': {},
        'prev_resp_act': 'api_call',
        'slots': [['food', 'irish']],
        'text': 'irish food'},
    {'prev_resp_act': 'canthelp_food', 'text': 'appleton wisconsin'},
    {'prev_resp_act': 'canthel

Для экономии времени и ресурсов мы будем использовать только 50 диалогов для обучения, но в реальных системах, конечно, требуется гораздо больше данных.


In [9]:
# скопируем
!cp my_data/simple-dstc2-trn.json my_data/simple-dstc2-trn.full.json

In [10]:

import json

NUM_TRAIN = 50 # число диалогов для обученния

with open('my_data/simple-dstc2-trn.full.json', 'rt') as fin:
    data = json.load(fin) # открываем и загружаем все
with open('my_data/simple-dstc2-trn.json', 'wt') as fout:
    json.dump(data[:NUM_TRAIN], fout, indent=2) # сохраняем только некоторые
print(f"Train set is reduced to {NUM_TRAIN} dialogues (out of {len(data)}).")

Train set is reduced to 50 dialogues (out of 967).


## 1. Создание базы данных

### База данных ресторанов

Наш чат-бот должен подбирать подходящий пользователю ресторан, надо создать базу данных о всех ресторанах, с которыми чат-бот работает.

Создадим базу данных `database`, которая будет содержать информацию о ресторанах: тип кухни, ценовой диапазон, адрес и др.

В базе данных информация представлена в виде таблиц, по которым можно делать поиск. 

Например, пользователь скажет "дешевая еда в южной части города".

Чат-бот должен понять, что слово "дешевая" означает  ценовой диапазон 'pricerange' со значением "cheap",
"в южной части города" означает место размещения 'area' ресторана со значением 'south'.

Тогда чат бот переведет запрос пользователя в набор некоторых полей и значений 'pricerange': 'cheap', 'area': 'south'.

Имея базу данных с такими полями мы сможем найти все подходящие записи. 

Такие поля, которые должен распознать чат-бот назвали "слотами" и теперь наша задача верно эти слоты распознавать. 

А уж потом искать в базе данных подходящие рестораны.

    >> database([{'pricerange': 'cheap', 'area': 'south'}])
    
    Out[1]: 
        [[{'name': 'the lucky star',
           'food': 'chinese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way cherry hinton',
           'phone': '01223 244277',
           'postcode': 'c.b 1, 7 d.y'},
          {'name': 'nandos',
           'food': 'portuguese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way',
           'phone': '01223 327908',
           'postcode': 'c.b 1, 7 d.y'}]]
           

&nbsp;
![gobot_database.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_database.png?raw=1)
&nbsp;

Обращение к базе данных ведется через api-запросы в специальном формате, значит чат-бот должен такие запросы формировать. В обучающих данных запросы содержатся в поле `"db_result"`.

In [11]:
!head -n 78 my_data/simple-dstc2-trn.json | tail +51

    {
      "speaker": 2,
      "text": "api_call area=\"south\" food=\"dontcare\" pricerange=\"cheap\"",
      "db_result": {
        "food": "chinese",
        "pricerange": "cheap",
        "area": "south",
        "addr": "cambridge leisure park clifton way cherry hinton",
        "phone": "01223 244277",
        "postcode": "c.b 1, 7 d.y",
        "name": "the lucky star"
      },
      "slots": [
        [
          "area",
          "south"
        ],
        [
          "pricerange",
          "cheap"
        ],
        [
          "food",
          "dontcare"
        ]
      ],
      "act": "api_call"
    },


Создадим базу данных, ключ  `primary_keys` указывает на главный слот с уникальными значениями, в нашем примере это название ресторана "name".  Будет создана пока пустая база данных в формате SQL.

In [12]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(primary_keys=["name"], # главный ключ 
                           save_path="my_bot/db.sqlite") # файл базы данных

2021-08-16 17:05:42.243 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2021-08-16 17:05:42.246 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 70: Initializing empty database on /content/my_bot/db.sqlite.



Теперь найдем в обучающих данных все `"db_result"` запросы и добавим их в базу данных:

In [17]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'): # перебираем все пакеты
    turns_x, turns_y = dialog # нам нужны только ответы системы
    db_results.extend( # собираем список
        x['db_result'] # из записей с запросами
        for x in turns_x[0] # из ответов системы
        if x.get('db_result')) # если в них есть такое поле

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results) # добавляем в базу данных

Adding 3016 items.


### Взаимодействие с базой данных

Теперь можно обращаться к базе данных. Найдем все дешевые рестораны на юге:

In [18]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'addr': 'cambridge leisure park clifton way',
   'area': 'south',
   'food': 'portuguese',
   'name': 'nandos',
   'phone': '01223 327908',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap'},
  {'addr': 'cambridge leisure park clifton way cherry hinton',
   'area': 'south',
   'food': 'chinese',
   'name': 'the lucky star',
   'phone': '01223 244277',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap'}]]

In [27]:
# файл с базой данных
!ls my_bot

db.sqlite  slotfill


В базе есть поля, по которым мы можем искать: 
- 'addr' - адрес
- 'area': район,
- 'food': тип кухни,
- 'name': название ресторана,
- 'phone': телефон,
- 'postcode': почтовый код,
- 'pricerange': ценовой диапазон.

## Определение слотов

Чтобы можно было искать в базе данных мы прежде должны распознать (говорят заполнить) слоты из вопроса пользователя.

Это сама по себе сложная задача, тут можно применять разные подходы. Самое банальное - сделать список всевозможных вариантов.

Для создания такого простого заполнителя слотов `Slot Filler` достаточно предоставить в json-файле `slot_vals.json` информацию о 
 - **slot types** - тип слота,
 - **slot values** - возможные значения слота,
 - примеры написания значений.

Пример:
Слот типа кухни 'food', может принимать значения 'chinese' (китайская), 'french' (французская) или 'dontcare' (все равно). Для каждого значения приведены возможные написания. 

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                
Здесь используется простой необучаемый заполнитель слотов, который сравнивает строки по [расстоянию Левенштейна](https://ru.wikipedia.org/wiki/Расстояние_Левенштейна).

Заполнитель слотов будет проверять ввод пользователя, сравнивать с возможными значениями, и выберет лучше всего подходящее значение слота. 

Более сложные заполнители могут автоматически учитывать разные варианты написания. 


&nbsp;
![gobot_slotfiller.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_slotfiller.png?raw=1)
&nbsp;

Пример использования `Slot Filler`:

    >> slot_filler(['I would like some chineese food'])
    
    Out[1]: [{'food': 'chinese'}]


Скачаем такой файл с описанием слотов.

In [28]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2021-08-16 17:33:09.240 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 444kB/s]
2021-08-16 17:33:10.988 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [29]:
!ls my_bot/slotfill

dstc_slot_vals.json


Напечатаем несколько типов слотов `slot types` и их значений `slot values` из файла.

In [30]:
!head -n 10 my_bot/slotfill/dstc_slot_vals.json

{
    "food": {
        "caribbean": [
            "carraibean",
            "carribean",
            "caribbean"
        ],
        "kosher": [
            "kosher"
        ],


Создадим конфигуратор заполнителя слотов из json-файла.

In [31]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

# создаем конфиг. заполнителя слотов по умолчанию
slotfill_config = read_json(configs.ner.slotfill_simple_dstc2_raw)

Создается конфиг. заполнителя слотов  из [DSTC2 slot-filling](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/ner/slotfill_dstc2_raw.json), заменим в нем пути к файлам на наши.

In [34]:
slotfill_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

Создадим заполнитель из его конфигурации.

In [36]:
from deeppavlov import build_model

slotfill = build_model(slotfill_config)

Проверьте  его работу.


In [48]:
slotfill(['i want cheap chinee food'])

[{'food': 'chinese', 'pricerange': 'cheap'}]

In [49]:
slotfill(['russian or chinese food'])

[{'food': 'russian'}]

Как видите, он довольно глупенький, ведь это просто сравниватель строк.
 
Можно проверить его работу на тестовых данных.

In [50]:
from deeppavlov import evaluate_model

slotfill = evaluate_model(slotfill_config);

2021-08-16 17:44:33.329 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my_data/simple-dstc2-trn.json]
2021-08-16 17:44:33.341 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my_data/simple-dstc2-val.json]
2021-08-16 17:44:33.394 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my_data/simple-dstc2-tst.json]
2021-08-16 17:44:33.625 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 479 samples in train split.
2021-08-16 17:44:33.626 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2021-08-16 17:44:33.627 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 samples in test split.


{"valid": {"eval_examples_count": 1444, "metrics": {"slots_accuracy": 0.9307}, "time_spent": "0:00:45"}}
{"test": {"eval_examples_count": 1386, "metrics": {"slots_accuracy": 0.9481}, "time_spent": "0:00:43"}}


Для хороших данных аккуратность довольно высока ~95% на тестовых данных.

Сохраним конфиг. на диск.

In [51]:
import json

json.dump(slotfill_config, open('my_bot/slotfill_config.json', 'wt'))

In [52]:
!ls my_bot

db.sqlite  slotfill  slotfill_config.json


## 3. Создание и обучение бота

### Правила диалога и шаблоны ответов

Цель чат-бота - как можно точней определить слоты, чтобы найти подходящий ресторан для пользователя. 

Но вот беда, пользователи частенько не сообщают всю нужную информацию или не знают чего хотят :).

Бот должен уметь поддержать диалог, переспросить, если не понятны некоторые важные слоты, уточнить информацию, поприветствовать и попрощаться с пользователем и т.п.

За ведение диалога отвечает специальный модуль "policy module" на основе нейронной сети.

В этом примере используется рекуррентная сеть с полносвязным слоем и активацией softmax, которая сохраняет предыдущее состояние, а значит знает, что пользователь отвечал ранее. 

Задача такой сети зная ввод пользователя и распознанные слоты, решить, какое действие предпринять. 

Действия могут быть различны: поприветствовать или попрощаться с пользователем, уточнить информацию (спросить о местонахождении, о предпочтениях в типе кухни и т.п.) или сделать запрос к базе данных и выдать результат.  
 

![gobot_policy.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_policy.png?raw=1)

Все возможные действия должны быть перечислены в файле `simple-dstc2-templates.txt`. Каждое же действие должно быть связано с шаблоном ответа, в который будут подставлены слоты.  

![gobot_templates.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_templates.png?raw=1)

Шаблоны ответов должны быть в формате `<act>TAB<template>`, где `<act>` - действие, а `<template>` - шаблон ответа.  

Шаблон ответа может содержать поля `#slot_type` в которые будет подставлены значения слотов из текущего диалога.

Такие шаблоны мы уже загрузили в файле simple-dstc2-templates.txt, посмотрим на него.

In [53]:
!head -n 10 my_data/simple-dstc2-templates.txt

api_call	api_call area="#area" food="#food" pricerange="#pricerange"
bye	You are welcome!
canthear	Sorry, I can't hear you.
canthelp_area	I'm sorry but there is no #area american restaurant in the #area of town.
canthelp_area_food	Sorry there is no #food restaurant in the #area of town.
canthelp_area_food_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #food food.
canthelp_area_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #area american food.
canthelp_food	I am sorry but there is no #food restaurant that matches your request.
canthelp_food_pricerange	Sorry there is no #food restaurant in the #pricerange price range.
confirm-domain	You are looking for a restaurant is that right?


Например, строка

```canthelp_area_food	Sorry there is no #food restaurant in the #area of town.```

означает действие, когда бот скажет пользователю, что в указанном районе нет ресторанов с указанной кухней.   

### Подготовка к обучению

Вообще-то задача модуля это задача классификации, классы это действия и они перечислены в `simple-dstc2-templates.txt`.

Так что для обучения нейронной сети нам нужна метка предпринятого действия в обучающих данных. В наборе DSTC-2 такие метки представлены в поле `"act"`.

In [55]:
!head -n 24 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },


Теперь сконфигурируем бота под наши данные и обучим его.

Возьмем общий файл конфигурации [simple DSTC2 bot config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot/gobot_simple_dstc2.json), {тут есть и [другие конфиги](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot) } из DeepPavlov и заменим в нем секции отвечающие за:
- embeddings - векторные представления слов из ввода пользователя, 
- database - базу данных ресторана,
- slot filler - заполнителя слотов,
- templates - шаблоны ответов,
- пути для загрузки\сохранения модели.

Загружаем бота:

In [56]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_simple_dstc2)

По умолчанию используется векторизатор на основе мешка слов (bag-of-words):

In [57]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

Конфигурируем под нашу базу данных:

In [58]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database', # тип базы данных
    'primary_keys': ["name"], # главный ключ
    'save_path': 'my_bot/db.sqlite' # путь к файлу базы данных
}

Конфигурируем под наш заполнитель слотов (на основе расстояния Левенштейна):

In [59]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_config.json'

Так как значения слотов могут уточняться в диалоге постепенно, то надо сохранять значения которые были получены ранее, отслеживать их.

Для этого применяется модуль `tracker`, который обновляет глобальное текущее состояние слотов (называют состояние диалога).  
Мы будем отслеживать только слоты:
- 'pricerange' - ценовой диапазон
- 'area' - район,
- 'food' - тип кухни,
- 'this' - слот, о котором идет речь в уточняющих вопросах системы. 

In [60]:
gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = ['pricerange', 'this', 'area', 'food']

Конфигурируем бота под наши шаблоны ответов:

In [61]:
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'my_data/simple-dstc2-templates.txt'

Конфигурируем пути к данным и модели:

In [62]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'
gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

Полностью диалоговая система выглядит так:

    
![gobot_pipeline.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_pipeline.png?raw=1)

### Обучение диалоговой системы

Мы готовы обучать нашу диалоговую систему. Осталось только задать параметры обучения и запустить его. Обучение рекуррентных сетей довольно долгое, в нашем примере ~30 минут, ждите. 

In [64]:
# чтоб не беспокоили предупреждения 
import tensorflow as tf
tf.get_logger().setLevel('INFO')

In [65]:
from deeppavlov import train_model # для обучения модели

gobot_config['train']['batch_size'] = 8 # размер пакета
gobot_config['train']['max_batches'] = 250 # максимальное количество пакетов
gobot_config['train']['val_every_n_batches'] = 40 # частота проверки
gobot_config['train']['log_every_n_batches'] = 40 # частота оценки на обучающих данных
# модель создается из конфига и обучается
train_model(gobot_config)

2021-08-16 18:44:26.751 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /root/.deeppavlov/downloads/dstc2_v3/dstc2-trn.jsonlist]
2021-08-16 18:44:27.306 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /root/.deeppavlov/downloads/dstc2_v3/dstc2-val.jsonlist]
2021-08-16 18:44:27.434 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /root/.deeppavlov/downloads/dstc2_v3/dstc2-tst.jsonlist]
2021-08-16 18:44:27.581 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/my_bot/word.dict]
2021-08-16 18:44:28.26 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/my_bot/word.dict]
2021-08-16 18:44:28.31 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for Sqlite3Database!
2021-08-16 18:44:28

INFO:tensorflow:Restoring parameters from /content/my_bot/model/policy


2021-08-16 18:46:43.256 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best per_item_dialog_accuracy of 0.0066


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0066}, "time_spent": "0:02:14", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 160, "metrics": {"per_item_dialog_accuracy": 0.4733}, "time_spent": "0:03:55", "epochs_done": 0, "batches_seen": 40, "train_examples_seen": 320, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.6503373607993126}}


2021-08-16 18:50:35.786 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 207: Improved best per_item_dialog_accuracy of 0.3598
2021-08-16 18:50:35.788 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 209: Saving model
2021-08-16 18:50:35.791 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/my_bot/model/policy]


...

2021-08-16 19:14:41.3 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/my_bot/word.dict]
2021-08-16 19:14:41.6 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2021-08-16 19:14:41.9 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /content/my_bot/db.sqlite.


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4837}, "time_spent": "0:02:10"}}


2021-08-16 19:14:42.517 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2021-08-16 19:14:42.522 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/my_bot/model/policy]


INFO:tensorflow:Restoring parameters from /content/my_bot/model/policy


Chainer[<deeppavlov.models.go_bot.wrapper.DialogComponentWrapper at 0x7f9ae39ddb90>,

### Оценка обучения

Оценим аккуратность (**accuracy**) обученного бота: совпадает ли ответ бота с ответами системы в тестовых данных (полное совпадение строк).

In [66]:
from deeppavlov import evaluate_model # для оценки

evaluate_model(gobot_config); # оцениваем

2021-08-16 19:14:56.475 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /root/.deeppavlov/downloads/dstc2_v3/dstc2-trn.jsonlist]
2021-08-16 19:14:56.645 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /root/.deeppavlov/downloads/dstc2_v3/dstc2-val.jsonlist]
2021-08-16 19:14:57.178 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /root/.deeppavlov/downloads/dstc2_v3/dstc2-tst.jsonlist]
2021-08-16 19:14:57.331 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/my_bot/word.dict]
2021-08-16 19:14:57.334 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2021-08-16 19:14:57.335 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from

INFO:tensorflow:Restoring parameters from /content/my_bot/model/policy
{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4908}, "time_spent": "0:02:11"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4837}, "time_spent": "0:02:07"}}


При обучении на `max_batches=250` пакетах аккуратность получислась`~ 0.5`. Не так хорошо, но и не так плохо.

## 4. Использование бота

Наш чат-бот готов.

Чтобы воспользоваться им надо создать модель в виде выполняемого объекта, а не файла конфигурации.

In [67]:
from deeppavlov import build_model # создание выполняемой модели 

bot = build_model(gobot_config) # создаем модель

2021-08-16 19:19:22.607 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/my_bot/word.dict]
2021-08-16 19:19:22.610 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2021-08-16 19:19:22.615 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /content/my_bot/db.sqlite.
2021-08-16 19:19:23.724 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2021-08-16 19:19:23.729 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/my_bot/model/policy]


INFO:tensorflow:Restoring parameters from /content/my_bot/model/policy


Работаем с ботом.

Прежде всего бот нас поприветствует (и проигнорирует наш вопрос, ха-ха).

In [68]:
bot(['hi, i want to eat, can you suggest a place to go?'])

[['Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?']]

Теперь можно его спросить о ресторанах.

In [69]:
bot(['i want cheap food'])

2021-08-16 19:19:31.934 INFO in 'deeppavlov.models.go_bot.tracker.dialogue_state_tracker'['dialogue_state_tracker'] at line 166: Made api_call with {'pricerange': 'cheap'}, got 22 results.


[['api_call area="dontcare" food="dontcare" pricerange="cheap"',
  'Thanh binh serves vietnamese food in the cheap price range.']]

Мы не сказали что любим китайскую еду, надо уточнить.

In [70]:
bot(['chinese food'])

2021-08-16 19:19:37.133 INFO in 'deeppavlov.models.go_bot.tracker.dialogue_state_tracker'['dialogue_state_tracker'] at line 166: Made api_call with {'pricerange': 'cheap', 'food': 'chinese'}, got 4 results.


[['api_call area="west" food="vietnamese" pricerange="cheap"',
  'The lucky star serves chinese food in the cheap price range.']]

Ладно, такой ресторан подойдет. Его адрес? 

In [71]:
bot(['thanks, give me their address'])

[['Sure, the lucky star is on cambridge leisure park clifton way cherry hinton.']]

А телефон?

In [72]:
bot(['i want their phone number too'])

[['The phone number of the lucky star is 01223 244277.']]

Ну все, пока.

In [73]:
bot(['bye'])

[['Can I help you with anything else?']]

Когда диалог закончен, необходимо сбросить состояние бота, иначе он будет думать, что мы продолжаем диалог, что может привести к дурацким ситуациям.

In [74]:
bot.reset()

In [75]:
bot(['hi, is there any cheap restaurant?'])

[['Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?']]

In [76]:
bot(['i want expensive indian food on north'])

2021-08-16 19:19:55.186 INFO in 'deeppavlov.models.go_bot.tracker.dialogue_state_tracker'['dialogue_state_tracker'] at line 166: Made api_call with {'pricerange': 'expensive', 'food': 'indian', 'area': 'north'}, got 1 results.


[['api_call area="north" food="indian" pricerange="expensive"',
  'Tandoori palace serves indian food in the expensive price range.']]

# Задания
Поиграйте с ботом. Придумайте вопросы, когда бот глупо ошибается. Придумайте вопросы когда бот молодец, отвечает на вопросы верно. Проверьте что будет, если не сбрасывать состояние бота. А если состояние сбрасывать после каждого вопроса? 

Можете ли ли вы улучшить бота?

# Ссылки

Использованы и адаптированы материалы:
* https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_extended_tutorial.ipynb 
* https://github.com/deepmipt/DeepPavlov



In [80]:
# Сохранить в архив, чтобы потом скачать себе на диск
!zip -r /content/my_bot.zip /content/my_bot

  adding: content/my_bot/ (stored 0%)
  adding: content/my_bot/word.dict (deflated 50%)
  adding: content/my_bot/slotfill_config.json (deflated 57%)
  adding: content/my_bot/model/ (stored 0%)
  adding: content/my_bot/model/policy.meta (deflated 88%)
  adding: content/my_bot/model/checkpoint (deflated 44%)
  adding: content/my_bot/model/policy.json (deflated 25%)
  adding: content/my_bot/model/policy.index (deflated 27%)
  adding: content/my_bot/model/policy.data-00000-of-00001 (deflated 7%)
  adding: content/my_bot/db.sqlite (deflated 78%)
  adding: content/my_bot/slotfill/ (stored 0%)
  adding: content/my_bot/slotfill/dstc_slot_vals.json (deflated 83%)


In [81]:
!zip -r /content/my_data.zip /content/my_data

  adding: content/my_data/ (stored 0%)
  adding: content/my_data/simple-dstc2-templates.txt (deflated 76%)
  adding: content/my_data/.done (stored 0%)
  adding: content/my_data/simple-dstc2-trn.full.json (deflated 94%)
  adding: content/my_data/simple-dstc2-val.json (deflated 94%)
  adding: content/my_data/simple-dstc2-trn.json (deflated 94%)
  adding: content/my_data/simple-dstc2-tst.json (deflated 94%)
